In [1]:
import os
import utils 
import sklearn
import re
import numpy as np
import librosa
from tqdm.notebook import tqdm


In [34]:
input_dir = r"C:\Users\Sulakshana\Desktop\STUDY\Course Work\thesis project\SER\audio_speech_actors_01-24"
file_extension = ".wav" 
file_paths, labels = utils.walk_filter(input_dir,file_extension)

In [35]:
zip(file_paths, labels)

In [36]:
window_len = 3.5
augment_factor = 1
sr = 16000

n_samples = int(window_len * sr)
mel_spec_list = []
audio_list = []
label_list = []
egmap_list = []
pitch_list = []
print("Number of files: ",len(file_paths))
for file,label in zip(file_paths,labels):
    signal = utils.load_audio(file,sr=16000)
    n_windows = int(signal.shape[0] // n_samples) * augment_factor
    n_correct_windows = 0
    # print("n_windows: ",n_windows)
    for i in range(n_windows):
        audio_frames = signal[ int(n_samples * i / augment_factor):int(n_samples * (1 + i / augment_factor))]
        audio_list.append(audio_frames)
        label_list.append(label)
        mel_spec_list.append(utils.extract_mel_spectogram(audio_frames))
        # pitch,egmaps= utils.extract_features_from_file(audio_frames)
        # egmap_list.append(egmaps)
        # pitch_list.append(pitch)



audio_arr = np.array(audio_list)      
label_arr = np.array(label_list) 
mel_spec_arr = np.array(mel_spec_list)
# egmaps_arr = np.array(egmap_list)
# pitch_arr = np.array(pitch_list)







Number of files:  1440


In [15]:
label_arr.shape,mel_spec_arr.shape#,egmaps_arr.shape,pitch_arr.shape

((106,), (106, 348, 64, 1), (106, 348, 25), (106, 348))

In [38]:
data_mel = mel_spec_arr.squeeze()
# pitch_arr_reshaped = pitch_arr[:,:,np.newaxis]
# data_mel_pitch = np.concatenate((data_mel,pitch_arr_reshaped),axis =2 )
# data_mel.shape,pitch_arr_reshaped.shape,data_mel_pitch.shape

In [39]:
number_of_datapoints = data_mel.shape[0]
indexes = np.random.permutation(number_of_datapoints)
train_idx = indexes[:int(number_of_datapoints*0.8)]
test_idx = indexes[int(number_of_datapoints*0.8):]
train_data = data_mel[train_idx]
test_data = data_mel[test_idx]
label_train = label_arr[train_idx]
label_test = label_arr[test_idx]
train_data.shape,test_data.shape,label_train.shape

((850, 348, 64), (213, 348, 64), (850,))

In [40]:
test_data.shape

(213, 348, 64)

In [41]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
n_train =train_data.shape[0]
n_test = test_data.shape[0]

clf.fit(train_data.reshape(n_train,-1),label_train)
y_pred = clf.predict(test_data.reshape(n_test,-1))

In [42]:
np.mean(y_pred == label_test)

0.38967136150234744

In [43]:
np.unique(label_test)

array([1, 2, 3, 4, 5, 6, 7, 8])

In [44]:
1/8

0.125

In [61]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from matplotlib.pyplot import specgram

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from tensorflow import keras
# import keras
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import optimizers
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(348,64)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
opt = optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [62]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 348, 128)          41088     
                                                                 
 activation_21 (Activation)  (None, 348, 128)          0         
                                                                 
 dropout_14 (Dropout)        (None, 348, 128)          0         
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 43, 128)          0         
 1D)                                                             
                                                                 
 conv1d_15 (Conv1D)          (None, 43, 128)           82048     
                                                                 
 activation_22 (Activation)  (None, 43, 128)           0         
                                                      

In [63]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [64]:
cnnhistory=model.fit(train_data, label_train, batch_size=16, epochs=30, validation_data=(test_data, label_test))


Epoch 1/30
54/54 [==============================] - 4s 52ms/step - loss: 2.6480 - accuracy: 0.1835 - val_loss: 2.5625 - val_accuracy: 0.2066
Epoch 2/30
54/54 [==============================] - 3s 48ms/step - loss: 1.9775 - accuracy: 0.2835 - val_loss: 1.9920 - val_accuracy: 0.2629
Epoch 3/30
54/54 [==============================] - 3s 47ms/step - loss: 1.8071 - accuracy: 0.3153 - val_loss: 1.8178 - val_accuracy: 0.3333
Epoch 4/30
54/54 [==============================] - 2s 46ms/step - loss: 1.6067 - accuracy: 0.3812 - val_loss: 1.7443 - val_accuracy: 0.3662
Epoch 5/30
54/54 [==============================] - 3s 62ms/step - loss: 1.4997 - accuracy: 0.4329 - val_loss: 1.7969 - val_accuracy: 0.2911
Epoch 6/30
54/54 [==============================] - 3s 61ms/step - loss: 1.3861 - accuracy: 0.4871 - val_loss: 1.7101 - val_accuracy: 0.3521
Epoch 7/30
54/54 [==============================] - 3s 59ms/step - loss: 1.3182 - accuracy: 0.5071 - val_loss: 1.6566 - val_accuracy: 0.4131
Epoch 8/30
54